Data Cleaning Codes

In [3]:
# importing the zipfile module
from zipfile import ZipFile
import pandas as pd

# loading the 2024_05_28 atlas_antibiotics.zip and creating a zip object
with ZipFile("/content/drive/MyDrive/2024_05_28 atlas_antibiotics.zip", 'r') as zObject:

	# Extracting all the members of the zip
	# into a specific location.
	zObject.extractall()

In [ ]:
# Read raw data and linealize antibiotics columnn

df=pd.read_csv('/content/2024_05_28 atlas_antibiotics.csv')

new_list=[]
for i in range(df.shape[0]):
  new_list.append(df.loc[i,'Amikacin': 'Tebipenem_I'].dropna().to_dict())

df=df.loc[:, :'Phenotype']
df['Antibiotics']= new_list

big_list=[]
for i in range(df.shape[0]):
  for row in df['Antibiotics'][i].keys():
    if row.endswith('_I'):
      x=row.split('_')[0]
      big_list.append([i, x, df['Antibiotics'][i][row], df['Antibiotics'][i][x] ])

Antibiotics=pd.DataFrame(big_list)

clean_df=df.merge(Antibiotics, left_index=True, right_on=0, how='left').drop(['Antibiotics', 0], axis=1).rename(columns={1:"Antibiotics", 2:"Status", 3:"COncentration"})

clean_df.to_csv("data.csv", sep=',', index=False)
# clean_df.to_csv("/content/drive/MyDrive/atlas_2024.csv", sep=',', index=False)

In [ ]:
# Join linealized data with gene data

df=pd.read_csv('/content/2024_05_28 atlas_antibiotics.csv')
df_=df.loc[:,'AMPC': 'GIM']
df_["Isolate Id"]= df["Isolate Id"]

clean_df.merge(df_, left_on="Isolate Id", right_on="Isolate Id", how="right").to_csv("/content/drive/MyDrive/atlas_gene_2024.csv", sep=',', index=False)

In [ ]:
# Extract rows with at least one gene present
atlas = pd.read_csv('2024_05_28 atlas_antibiotics.csv',low_memory=False)
atlas.dropna(subset=['AMPC', 'SHV', 'TEM', 'CTXM1', 'CTXM2', 'CTXM825', 'CTXM9', 'VEB', 'PER', 'GES', 'ACC', \
                     'CMY1MOX', 'CMY11', 'DHA', 'FOX', 'ACTMIR', 'KPC', 'OXA', 'NDM', 'IMP', 'VIM', 'SPM', 'GIM'], how='all', inplace=True).to_csv("/content/drive/MyDrive/genotypes_atlas.csv")


In [ ]:
# Linealize Gene columns

df=pd.read_csv("/content/drive/MyDrive/genotypes_atlas.csv")
df.drop(["Unnamed: 0", "Unnamed: 134"], axis=1, inplace=True)

df1=pd.DataFrame()

for i in range(54877):
  df_= df.loc[i, "AMPC":].dropna()
  df_= df_[df_ != "NEG"]
  df_= df_[df_ != "-"]
  df_=df_.to_frame().reset_index().rename(columns={"index": "Gene", i: "Strain"})
  df_['Index']=i
  df1=pd.concat([df1,df_])

df=df.loc[:, :'Phenotype']
df.merge(df1, left_index=True, right_on="Index", how="right").drop("Index", axis=1).to_csv("/content/drive/MyDrive/atlas_2024_genes_.csv", sep=',', index=False)

Gene Prediction

In [17]:
# Read data
import pandas as pd
df=pd.read_csv('/content/2024_05_28 atlas_antibiotics.csv')

<ipython-input-17-28fd8dd0c5b6>:3: DtypeWarning: Columns (5,10,19,20,23,24,29,30,35,36,43,44,45,46,55,56,57,58,59,61,62,63,64,65,67,68,69,70,71,72,73,74,75,76,77,78,79,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,101,102,103,104,105,106,107,109,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/2024_05_28 atlas_antibiotics.csv')


In [18]:
# Get columns with missing values
Ma=df.loc[:, "Amikacin":"Tebipenem_I"].isna().sum().to_frame()
Ma[0] = (Ma[0]/df.shape[0]) * 100
Ma=Ma.sort_values(0, ascending=False)

In [19]:
# Get list of columns with missing values
drop=list(Ma.index.values)[:62]

In [20]:
# Drop the columns
df1 = df.drop(drop, axis=1)

In [21]:
for i in df.loc[:, "AMPC":].columns:
  df_=df[i].dropna()
  df_= df_[df_ != "-"]
  df_.replace("NEG", 0, inplace=True)
  df_.where(df_.isin([0]), 1, inplace=True)

  print(df_.value_counts())

AMPC
0    10995
1      832
Name: count, dtype: int64
SHV
0    31629
1    21583
Name: count, dtype: int64
TEM
0    30218
1    23008
Name: count, dtype: int64
CTXM1
1    26170
0    16159
Name: count, dtype: int64
CTXM2
0    41671
1      626
Name: count, dtype: int64
CTXM825
0    42128
1      166
Name: count, dtype: int64
CTXM9
0    38138
1     4165
Name: count, dtype: int64
VEB
0    52420
1      455
Name: count, dtype: int64
PER
0    52705
1      170
Name: count, dtype: int64
GES
0    52377
1      498
Name: count, dtype: int64
ACC
0    42237
1       58
Name: count, dtype: int64
CMY1MOX
0    41777
1       14
Name: count, dtype: int64
CMY11
0    40137
1     1854
Name: count, dtype: int64
DHA
0    40955
1     1264
Name: count, dtype: int64
FOX
0    35522
1       33
Name: count, dtype: int64
ACTMIR
0    40879
1      214
Name: count, dtype: int64
KPC
0    49309
1     4151
Name: count, dtype: int64
OXA
0    43957
1     4195
Name: count, dtype: int64
NDM
0    50728
1     2646
Name: count, dtype

In [22]:
# Drop gene columns with very few POS genes present
df1=df1.drop(["GIM", 'SPM', 'IMP', 'ACTMIR', 'FOX', 'CMY1MOX', 'ACC', 'PER', 'CTXM825', 'Unnamed: 134'], axis=1)

In [23]:
# Function to clean MIC Values columns
def clean(x):
  x=str(x).strip(">=")
  x=str(x).strip("<")
  x=str(x).strip(">")
  x=str(x).strip("<=")
  return x

In [24]:
# clean MIC Values columns
for i in df1.loc[:, "Amikacin": "Trimethoprim sulfa"].columns:
  if i.endswith("_I"):
    continue
  else:
    df1[i]=df1[i].apply(clean)
    df1[i]=df1[i].astype("float64")

In [25]:
# Drop Unneeded columns
df1.drop(["Isolate Id", "Study",  "State", "Year"], axis=1, inplace=True)

In [26]:
# Rename Patient column
df1.rename(columns={'In / Out Patient': 'Patient'}, inplace=True)

In [27]:
# Label encode Categorical variable

from sklearn.preprocessing import LabelEncoder
import joblib

le=LabelEncoder()

for i in ["Species", "Family", "Country", "Gender", "Speciality", "Source", "Age Group", "Patient", "Phenotype"]:
  df1[i]=le.fit_transform(df1[i])

In [28]:
# Fillna with -999
df1.fillna(-999, inplace=True)

# Encode status column
df1.replace("Susceptible", 1, inplace=True)
df1.replace("Resistant", 0, inplace=True)
df1.replace("Intermediate", 2, inplace=True)

In [31]:
for i in ['AMPC', 'SHV', 'TEM', 'CTXM1', 'CTXM2', 'CTXM9', 'VEB', 'GES', 'CMY11', 'DHA', 'KPC', 'OXA', 'NDM', 'VIM']:

  df_=df1.loc[:, :i]
  df_ = df_[df_[i] != -999]
  df_=df_[df_[i] != "-"]
  df_[i]=df_[i].replace("NEG", 0)
  df_[i]=df_[i].where(df_[i].isin([0]), 1)

  y=df_[i]
  y=y.astype("int64")
  X=df_.loc[:, :"Trimethoprim sulfa"]

  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  from sklearn.ensemble import RandomForestClassifier
  rf=RandomForestClassifier()
  rf.fit(X_train, y_train)
  y_pred=rf.predict(X_test)

  from sklearn.metrics import f1_score
  print(i, rf.score(X_test, y_test), f1_score(y_test, y_pred))

AMPC 0.9633699633699634 0.7031963470319634
SHV 0.9674267100977199 0.9600798403193613
TEM 0.8075526052104208 0.7802017023102782
CTXM1 0.9171588314040475 0.9340273422801957
CTXM2 0.9879432624113476 0.4354243542435424
CTXM9 0.9405090221416752 0.6364949446316802
VEB 0.9965328122045011 0.7511312217194569
GES 0.9945155393053017 0.6329113924050632
CMY11 0.9740434989680902 0.6418400876232202
DHA 0.979709458392547 0.5301645338208409
KPC 0.9837885023070209 0.8876404494382022
OXA 0.9817942683095666 0.8952608522500995
NDM 0.9911946543433461 0.9120399251403618
VIM 0.985249621785174 0.7617107942973524


MIC/Susceptibility Prediction

In [ ]:
# Generate data with Meta-Data, Genes present and Antibiotics status Linealized
import pandas as pd

df=pd.read_csv("/content/drive/MyDrive/genotypes_atlas.csv")
df.drop(["Unnamed: 0", "Unnamed: 134"], axis=1, inplace=True)

genes=[]
index=[]

for i in range(54877):
  df_= df.loc[i, "AMPC":].dropna()
  df_= df_[df_ != "NEG"]
  df_= df_[df_ != "-"]
  df_=df_.to_frame().reset_index().rename(columns={"index": "Gene", i: "Strain"})
  gene=df_["Gene"].values
  genes.append(gene)
  index.append(i)

df1=pd.DataFrame()
df1["Gene"]=genes
df1["Index"]=index

df=df.loc[:, :'Phenotype']
df=df.merge(df1, left_index=True, right_on="Index").drop("Index", axis=1)
df=df[["Isolate Id", "Gene"]]
result=pd.get_dummies(df["Gene"].squeeze().explode()).groupby(level=0).sum()
result["Isolate Id"]=df["Isolate Id"]

df1=pd.read_csv("/content/drive/MyDrive/atlas_2024.csv")
result.merge(df1, left_on="Isolate Id", right_on="Isolate Id").to_csv("/content/drive/MyDrive/new_atlas_2024_genes_.csv", sep=',', index=False)

<ipython-input-3-56b16f73be9f>:2: DtypeWarning: Columns (56,57,66,70,71,72,73,76,77,78,79,82,83,90,96,97,98,99,100,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/content/drive/MyDrive/genotypes_atlas.csv")


In [32]:
# Read Data
df= pd.read_csv("/content/drive/MyDrive/new_atlas_2024_genes_.csv")
df

<ipython-input-32-6d353966cb37>:1: DtypeWarning: Columns (27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("/content/drive/MyDrive/new_atlas_2024_genes_.csv")


,ACC,ACTMIR,AMPC,CMY11,CMY1MOX,CTXM1,CTXM2,CTXM825,CTXM9,DHA,...,Gender,Age Group,Speciality,Source,In / Out Patient,Year,Phenotype,Antibiotics,Status,COncentration
0,0,0,0,0,0,0,0,0,1,0,...,Female,85 and Over,Medicine General,Urine,None Given,2013,ESBL,Amikacin,Susceptible,4
1,0,0,0,0,0,0,0,0,1,0,...,Female,85 and Over,Medicine General,Urine,None Given,2013,ESBL,Amoxycillin clavulanate,Susceptible,4
2,0,0,0,0,0,0,0,0,1,0,...,Female,85 and Over,Medicine General,Urine,None Given,2013,ESBL,Ampicillin,Resistant,>32
3,0,0,0,0,0,0,0,0,1,0,...,Female,85 and Over,Medicine General,Urine,None Given,2013,ESBL,Cefepime,Resistant,>32
4,0,0,0,0,0,0,0,0,1,0,...,Female,85 and Over,Medicine General,Urine,None Given,2013,ESBL,Ceftazidime,Susceptible,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815216,0,0,0,0,0,1,0,0,0,0,...,NaN,19 to 64 Years,Medicine General,Blood,None Given,2013,ESBL,Levofloxacin,Resistant,>8
815217,0,0,0,0,0,1,0,0,0,0,...,NaN,19 to 64 Years,Medicine General,Blood,None Given,2013,ESBL,Meropenem,Susceptible,<=0.06
815218,0,0,0,0,0,1,0,0,0,0,...,NaN,19 to 64 Years,Medicine General,Blood,None Given,2013,ESBL,Minocycline,Intermediate,8
815219,0,0,0,0,0,1,0,0,0,0,...,NaN,19 to 64 Years,Medicine General,Blood,None Given,2013,ESBL,Piperacillin tazobactam,Susceptible,2


In [33]:
# Drop Unnecessary Columns
df.drop(['Isolate Id','Study', 'State','COncentration', "Year"], axis=1, inplace=True)

In [34]:
df.drop("Phenotype", axis=1, inplace=True)

In [35]:
# Fill Missing Values
df['Gender'].fillna('Male', inplace=True)
df['In / Out Patient'].fillna('Inpatient', inplace=True)
df['In / Out Patient']=df['In / Out Patient'].str.replace('None Given', 'Other')

In [36]:
# Rename Patient column
df.rename(columns={'In / Out Patient': 'Patient'}, inplace=True)

In [37]:
# Label encode Categorical Columns

cols=['Species', 'Family', 'Country', 'Gender', 'Age Group', 'Speciality',
       'Source', 'Patient', 'Antibiotics']

from sklearn import preprocessing
import pickle
for i in cols:
  le = preprocessing.LabelEncoder()
  df[i]=le.fit_transform(df[i])


In [38]:
# Encode Target Columm

df['Status'].replace('Susceptible', 0, inplace=True)
df['Status'].replace('Resistant', 1, inplace=True)
df['Status'].replace('Intermediate', 2, inplace=True)

In [39]:
# Light GBM Model

X=df.drop('Status', axis=1)
Y=df['Status']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.3, random_state=42)

import lightgbm
r= lightgbm.LGBMClassifier()
r.fit(X_train, y_train)
r.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.083443 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 279
[LightGBM] [Info] Number of data points in the train set: 570654, number of used features: 31
[LightGBM] [Info] Start training from score -0.953010
[LightGBM] [Info] Start training from score -0.712240
[LightGBM] [Info] Start training from score -2.088465


0.8180048820977482

Prediction Function

In [40]:
# Drop columns
drop= ['Tebipenem_I',
 'Ceftibuten avibactam_I',
 'Aztreonam avibactam_I',
 'Ceftaroline avibactam_I',
 'Cefoperazone sulbactam_I',
 'Sulbactam_I',
 'Gatifloxacin_I',
 'Tetracycline_I',
 'Tetracycline',
 'Gatifloxacin',
 'Quinupristin dalfopristin',
 'Quinupristin dalfopristin_I',
 'Sulbactam',
 'Cefoxitin',
 'Cefoxitin_I',
 'Metronidazole',
 'Metronidazole_I',
 'Clarithromycin',
 'Clarithromycin_I',
 'Cefpodoxime_I',
 'Ceftibuten_I',
 'Cefixime_I',
 'Azithromycin_I',
 'Azithromycin',
 'Ertapenem_I',
 'Tebipenem',
 'Cefpodoxime',
 'Ceftibuten avibactam',
 'Ceftibuten',
 'Cefixime',
 'Ertapenem',
 'Meropenem vaborbactam_I',
 'Moxifloxacin_I',
 'Moxifloxacin',
 'Ceftaroline avibactam',
 'Meropenem vaborbactam',
 'Ceftolozane tazobactam_I',
 'Oxacillin_I',
 'Penicillin_I',
 'Ceftolozane tazobactam',
 'Doripenem_I',
 'Ampicillin sulbactam_I',
 'Oxacillin',
 'Teicoplanin_I',
 'Daptomycin_I',
 'Teicoplanin',
 'Ciprofloxacin_I',
 'Ciprofloxacin',
 'Daptomycin',
 'Penicillin',
 'Cefoperazone sulbactam',
 'Clindamycin_I',
 'Erythromycin_I',
 'Doripenem',
 'Erythromycin',
 'Clindamycin',
 'Gentamicin_I',
 'Trimethoprim sulfa_I',
 'Ceftazidime avibactam_I',
 'Colistin_I',
 'Colistin',
 'Aztreonam_I']

In [41]:
import numpy as np
import pandas as pd


# Sample data
data = {
    'Species': 'Pseudomonas aeruginosa',
    'Family': 'Non-Enterobacteriaceae',
    'Country': 'Argentina',
    'Gender': 'Male',
    'Age Group': '65 to 84 Years',
    'Speciality': 'Other',
    'Source': 'Ulcer',
    'Patient': 'None Given',
    'Phenotype': np.nan,
    'Amikacin': 32,
    'Amikacin_I': 'Resistant',
    'Amoxycillin clavulanate': 32,
    'Amoxycillin clavulanate_I': np.nan,
    'Ampicillin': 32,
    'Ampicillin_I': np.nan,
    'Azithromycin': np.nan,
    'Azithromycin_I': np.nan,
    'Cefepime': 16,
    'Cefepime_I': 'Resistant',
    'Cefoxitin': np.nan,
    'Cefoxitin_I': np.nan,
    'Ceftazidime': 64,
    'Ceftazidime_I': 'Resistant',
    'Ceftriaxone': np.nan,
    'Ceftriaxone_I': np.nan,
    'Clarithromycin': np.nan,
    'Clarithromycin_I': np.nan,
    'Clindamycin': np.nan,
    'Clindamycin_I': np.nan,
    'Erythromycin': np.nan,
    'Erythromycin_I': np.nan,
    'Imipenem': 2,
    'Imipenem_I': 'Susceptible',
    'Levofloxacin': 8,
    'Levofloxacin_I': 'Resistant',
    'Linezolid': np.nan,
    'Linezolid_I': np.nan,
    'Meropenem': 8,
    'Meropenem_I': 'Resistant',
    'Metronidazole': np.nan,
    'Metronidazole_I': np.nan,
    'Minocycline': np.nan,
    'Minocycline_I': np.nan,
    'Penicillin': np.nan,
    'Penicillin_I': np.nan,
    'Piperacillin tazobactam': 128,
    'Piperacillin tazobactam_I': 'Resistant',
    'Tigecycline': 8,
    'Tigecycline_I': np.nan,
    'Vancomycin': np.nan,
    'Vancomycin_I': np.nan,
    'Ampicillin sulbactam': np.nan,
    'Ampicillin sulbactam_I': np.nan,
    'Aztreonam': 32,
    'Aztreonam_I': 'Resistant',
    'Aztreonam avibactam': 32,
    'Aztreonam avibactam_I': np.nan,
    'Cefixime': np.nan,
    'Cefixime_I': np.nan,
    'Ceftaroline': 128,
    'Ceftaroline_I': np.nan,
    'Ceftaroline avibactam': np.nan,
    'Ceftaroline avibactam_I': np.nan,
    'Ceftazidime avibactam': 8,
    'Ceftazidime avibactam_I': 'Susceptible',
    'Ciprofloxacin': np.nan,
    'Ciprofloxacin_I': np.nan,
    'Colistin': 1,
    'Colistin_I': 'Intermediate',
    'Daptomycin': np.nan,
    'Daptomycin_I': np.nan,
    'Doripenem': 8,
    'Doripenem_I': 'Resistant',
    'Ertapenem': np.nan,
    'Ertapenem_I': np.nan,
    'Gatifloxacin': np.nan,
    'Gatifloxacin_I': np.nan,
    'Gentamicin': np.nan,
    'Gentamicin_I': np.nan,
    'Moxifloxacin': np.nan,
    'Moxifloxacin_I': np.nan,
    'Oxacillin': np.nan,
    'Oxacillin_I': np.nan,
    'Quinupristin dalfopristin': np.nan,
    'Quinupristin dalfopristin_I': np.nan,
    'Sulbactam': np.nan,
    'Sulbactam_I': np.nan,
    'Teicoplanin': np.nan,
    'Teicoplanin_I': np.nan,
    'Tetracycline': np.nan,
    'Tetracycline_I': np.nan,
    'Trimethoprim sulfa': np.nan,
    'Trimethoprim sulfa_I': np.nan,
    'Ceftolozane tazobactam': np.nan,
    'Ceftolozane tazobactam_I': np.nan,
    'Cefoperazone sulbactam': np.nan,
    'Cefoperazone sulbactam_I': np.nan,
    'Meropenem vaborbactam': np.nan,
    'Meropenem vaborbactam_I': np.nan,
    'Cefpodoxime': np.nan,
    'Cefpodoxime_I': np.nan,
    'Ceftibuten': np.nan,
    'Ceftibuten_I': np.nan,
    'Ceftibuten avibactam': np.nan,
    'Ceftibuten avibactam_I': np.nan,
    'Tebipenem': np.nan,
    'Tebipenem_I': np.nan
}
# Convert to DataFrame
df = pd.DataFrame([data])

In [42]:
df.drop(drop, axis=1, inplace=True)
df.columns

Index(['Species', 'Family', 'Country', 'Gender', 'Age Group', 'Speciality',
       'Source', 'Patient', 'Phenotype', 'Amikacin', 'Amikacin_I',
       'Amoxycillin clavulanate', 'Amoxycillin clavulanate_I', 'Ampicillin',
       'Ampicillin_I', 'Cefepime', 'Cefepime_I', 'Ceftazidime',
       'Ceftazidime_I', 'Ceftriaxone', 'Ceftriaxone_I', 'Imipenem',
       'Imipenem_I', 'Levofloxacin', 'Levofloxacin_I', 'Linezolid',
       'Linezolid_I', 'Meropenem', 'Meropenem_I', 'Minocycline',
       'Minocycline_I', 'Piperacillin tazobactam', 'Piperacillin tazobactam_I',
       'Tigecycline', 'Tigecycline_I', 'Vancomycin', 'Vancomycin_I',
       'Ampicillin sulbactam', 'Aztreonam', 'Aztreonam avibactam',
       'Ceftaroline', 'Ceftaroline_I', 'Ceftazidime avibactam', 'Gentamicin',
       'Trimethoprim sulfa'],
      dtype='object')

In [43]:
# Gene Prediction
import joblib
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

lst=[]

def predict_gene(df):
  for i in ["Species", "Family", "Country", "Gender", "Speciality", "Source", "Age Group", "Patient", "Phenotype"]:
    encoder="/content/drive/MyDrive/packages_/"+i+".joblib"
    le=joblib.load(encoder)
    df[i]=le.transform(df[i])
  df.fillna(-999, inplace=True)
  df.replace("Susceptible", 1, inplace=True)
  df.replace("Resistant", 0, inplace=True)
  df.replace("Intermediate", 2, inplace=True)
  #df.drop(drop, axis=1, inplace=True)
  for i in ['AMPC', 'SHV', 'TEM', 'CTXM1', 'CTXM2', 'CTXM9', 'VEB', 'GES', 'CMY11', 'DHA', 'KPC', 'OXA', 'NDM', 'VIM']:
    name="/content/drive/MyDrive/packages_/"+i+".joblib"
    rf=joblib.load(name)
    pred=rf.predict(df.values)
    pred_=rf.predict_proba(df.values)
    lst.append(pred_[0])
    print(i, pred)
  s=['AMPC', 'SHV', 'TEM', 'CTXM1', 'CTXM2', 'CTXM9', 'VEB', 'GES', 'CMY11', 'DHA', 'KPC', 'OXA', 'NDM', 'VIM']
  df2=pd.DataFrame(lst)
  df2["Genes"]=s
  df2.rename(columns={0:"NEG", 1:"POS"}, inplace=True)
  fig=px.bar(
  data_frame = df2,
  x = "Genes",
  y = ["NEG","POS"],
  opacity = 0.9,
  orientation = "v",
  barmode = 'group',
  title='')

  return (fig.show())

predict_gene(df)

AMPC [0]
SHV [0]
TEM [0]
CTXM1 [0]
CTXM2 [0]
CTXM9 [0]
VEB [0]
GES [0]
CMY11 [0]
DHA [0]
KPC [0]
OXA [0]
NDM [0]
VIM [0]


In [44]:
# Mic Prediction

import pickle

data = {'ACC': 0,
 'ACTMIR': 0,
 'AMPC': 0,
 'CMY11': 0,
 'CMY1MOX': 0,
 'CTXM1': 1,
 'CTXM2': 0,
 'CTXM825': 0,
 'CTXM9': 0,
 'DHA': 0,
 'FOX': 0,
 'GES': 0,
 'IMP': 0,
 'KPC': 0,
 'NDM': 0,
 'OXA': 0,
 'PER': 0,
 'SHV': 0,
 'SPM': 0,
 'TEM': 0,
 'VEB': 0,
 'VIM': 0,

 'Species': 'Escherichia coli',
 'Family': 'Enterobacteriaceae',
 'Country': 'Spain',

 'Gender': 'Female',
 'Age Group': '65 to 84 Years',
 'Speciality': 'Emergency Room',
 'Source': 'Urine',
 'Patient': 'Outpatient',
 'Antibiotics': 'Amoxycillin clavulanate'}

df=pd.DataFrame([data])

def Predict_MIC(df):
  df['Gender'].fillna('Male', inplace=True)
  df['Patient'].fillna('Inpatient', inplace=True)
  df['Patient']=df['Patient'].str.replace('None Given', 'Other')

  for i in ["Species", "Family", "Country", "Gender", "Speciality", "Source", "Age Group", "Patient", "Antibiotics"]:
    encoder="/content/drive/MyDrive/packages/"+i+".pkl"
    file_ = open(encoder,'rb')
    le = pickle.load(file_)
    df[i]=le.transform(df[i])

  name="/content/drive/MyDrive/packages/status.joblib"
  r=joblib.load(name)
  pred=r.predict(df.values)
  pred_=r.predict_proba(df.values)

  if pred==0:
    print("Susceptible")
  elif pred==1:
    print("Resistant")
  else:
    print("Intermediate")

  fig = px.pie(values=pred_[0], names=["Susceptible", "Resistant", "Intermediate"])
  return(fig.show())

Predict_MIC(df)

Intermediate
